<a href="https://colab.research.google.com/github/rajdeepd/bpb-vector-databases/blob/main/chapter10/evaluate_llama_index_indian_budget.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install -q llama-index

In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules
IN_COLAB

False

In [2]:
if IN_COLAB:
    !pip install -q llama-index

In [3]:
import nest_asyncio

nest_asyncio.apply()

from llama_index.core.evaluation import generate_question_context_pairs
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.evaluation import generate_question_context_pairs
from llama_index.core.evaluation import RetrieverEvaluator
from llama_index.llms.openai import OpenAI

import os
import pandas as pd

In [4]:
OPENAI_API_KEY = ''
if IN_COLAB:
    from google.colab import userdata
    OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
    len(OPENAI_API_KEY)
else:
    from jproperties import Properties

    configs = Properties()
    with open('../config.properties', 'rb') as config_file:
        configs.load(config_file)
        print(configs.get("OPENAI_API_KEY").data[:5]) 
        OPENAI_API_KEY = configs.get("OPENAI_API_KEY").data

sk-pr


# Setup the OpenAI API key
We will setup the OPENAI_API_KEY in the environment variable. This will be used by LlamaIndex as we have seen earier. We are also checking validity of the key 

In [46]:
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
#OPENAI_API_KEY
len(os.environ['OPENAI_API_KEY'])

164

In [47]:
import openai

def check_openai_api_key(api_key):
    client = openai.OpenAI(api_key=api_key)
    try:
    t except openai.AuthenticationError:
        return False
    else:
        return True



if check_openai_api_key(OPENAI_API_KEY):
    print("It is a valid OpenAI API key.")

It is a valid OpenAI API key.


# **Indian Budget Speech from 2025.**

**Load Data**
We will check if the notebook is running in Google colab or not with a flag.

In [7]:
if IN_COLAB:
    # Load the Drive helper and mount
    from google.colab import drive
    
    # This will prompt for authorization.
    drive.mount('/content/drive')

Based on the flag load appropriate path into `PATH` variable.

In [48]:
PATH= ''
if IN_COLAB:
    #!ls "/content/indian_budget"
    PATH="/content/indian_budget"
else:
    PATH='../data/indian_budget'
# Check that PATH is showing the markdown we are interested in
!ls $PATH

indian_budget_speech_2025.md


# **Load Data and Build Index.**

In [49]:
documents = SimpleDirectoryReader(PATH).load_data()

# Define an LLM we want to use, gpt-4 in thsi case.
llm = OpenAI(model="gpt-4")

# Build the vectorindex with a chunk_size of 512
node_parser = SimpleNodeParser.from_defaults(chunk_size=512)
nodes = node_parser.get_nodes_from_documents(documents)
vector_index = VectorStoreIndex(nodes)

In [50]:
len(nodes)

29

We have 29 nodes loaded from the document.As we learnt earlier each node is a chunk from previous document.

# **Build a QueryEngine and Query.**
We will build query engine which can take plain text queries and try to find closest responses.

In [51]:

query_engine = vector_index.as_query_engine()

# Query Engine by default retrieves two similar nodes/ chunks. It can be modify using following APIs
# vector_index.as_query_engine(similarity_top_k=k).


response_vector_1 = query_engine.query("What is the  revised estimate of total expenditure?")
response_vector_2 = query_engine.query("What is total fiscal deficit")

In [52]:
len(response_vector_1.source_nodes)

2

**print and check the response**

In [53]:
response_vector_1.response

'The revised estimate of total expenditure is ₹47.16 lakh crore.'

In [54]:
response_vector_2.response

'The total fiscal deficit is estimated to be 4.4 per cent of GDP.'

In [55]:
# Check number of chunks retrieved.
len(response_vector.source_nodes)

2

# Build Evaluation Data Set
## Question-Context Pair Generation
Let us now build a simple evaluation dataset over the existing text corpus we created above

We use method generate_question_context_pairs to generate a set of (question, context) pairs over the unstructured text corpus. This method uses the LLM to auto-generate questions from each context chunk.

the method returns  a EmbeddingQAFinetuneDataset object. Tthis contains at a high level  a set of ids mapping to queries and relevant chunks and the corpus

In [18]:
qa_dataset = generate_question_context_pairs(
    nodes,
    llm=llm,
    num_questions_per_chunk=2
)

100%|████████████████████████████████████████████████████████████████████████████████| 29/29 [01:33<00:00,  3.22s/it]


Having built the dataset for evaluation we will not create a retriever and evaluator.

# **Retrieval Evaluation:**
Retrievers help fetch the most relevant context based on the user query. We will build it based on the index created.

In [19]:
retriever = vector_index.as_retriever(similarity_top_k=2)

# **Mean Reciprocal Rank (MRR):**

In [20]:
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=retriever
)

In [21]:
# Evaluate
eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset)

In [22]:
def display_results(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()

    metric_df = pd.DataFrame(
        {"Retriever Name": [name], "Hit Rate": [hit_rate], "MRR": [mrr]}
    )

    return metric_df

In [23]:
display_results("OpenAI Embedding Retriever", eval_results)

,Retriever Name,Hit Rate,MRR
0,OpenAI Embedding Retriever,0.931034,0.775862


# **Response Evaluation**

In [24]:
# Get the list of queries from the above created dataset

queries = list(qa_dataset.queries.values())

In [56]:
queries[:5]

["In the Budget 2025-2026 speech by Nirmala Sitharaman, what are the five key objectives that the Government's efforts are focused on, as mentioned in the introduction?",
 "According to the table of contents, what are the four 'engines' and the 'fuel' that are discussed in Part A of the budget speech?",
 "What are the five key objectives of the Government's budget as mentioned in the introduction of the document?",
 "According to the document, what are the six aspects that encompass 'Viksit Bharat' as per the great Telugu poet and playwright Gurajada Appa Rao's perspective?",
 '"What are the six domains that the Budget aims to initiate transformative reforms in over the next five years, according to the context provided?"']

# **Faithfulness Evaluator**

In [26]:
# gpt-3.5-turbo
#gpt35 = OpenAI(temperature=0, model="gpt-3.5-turbo")
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.openai import OpenAI

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
#Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
#Settings.num_output = 512
#Settings.context_window = 3900
##service_context_gpt35 = ServiceContext.from_defaults(llm=gpt35)

# gpt-3.5-turbo-16k-0613
gpt4 = OpenAI(temperature=0, model="gpt-4")
#service_context_gpt4 = ServiceContext.from_defaults(llm=gpt4)

#gpt-4
#gpt35T = OpenAI(temperature=0, model="gpt-4")
#service_context_gpt4 = ServiceContext.from_defaults(llm=gpt35T)

In [27]:
#Create a QueryEngine with gpt-3.5-turbo service_context to generate response for the query.
#vector_index = VectorStoreIndex(nodes, service_context = service_context_gpt35)
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()
#query_engine = VectorStoreIndex.as_query_engine()

In [28]:
#Create a FaithfulnessEvaluator
from llama_index.core.evaluation import FaithfulnessEvaluator
#faithfulness_gpt4 = FaithfulnessEvaluator(service_context=service_context_gpt4)
faithfulness_gpt4  = FaithfulnessEvaluator(llm=gpt4)


In [29]:
eval_query = queries[1]

eval_query

"According to the table of contents, what are the four 'engines' and the 'fuel' that are discussed in Part A of the budget speech?"

In [30]:
#Generate response first and use faithfull evaluator.
response_vector = query_engine.query(eval_query)

In [31]:
# Compute faithfulness evaluation

eval_result = faithfulness_gpt4.evaluate_response(response=response_vector)

In [32]:
# You can check passing parameter in eval_result if it passed the evaluation.
eval_result.passing

True

# **Relevancy Evaluator**

In [33]:
from llama_index.core.evaluation import RelevancyEvaluator

#relevancy_gpt4 = RelevancyEvaluator(service_context=service_context_gpt4)
relevancy_gpt4 = RelevancyEvaluator(llm=gpt4)

In [34]:
# Pick a query
query = queries[1]

query

"According to the table of contents, what are the four 'engines' and the 'fuel' that are discussed in Part A of the budget speech?"

In [35]:
# Generate response.
# response_vector has response and source nodes (retrieved context)
response_vector = query_engine.query(query)

# Relevancy evaluation
eval_result = relevancy_gpt4.evaluate_response(
    query=query, response=response_vector
)

In [36]:
# You can check passing parameter in eval_result if it passed the evaluation.
eval_result.passing

True

In [37]:
# You can get the feedback for the evaluation.
eval_result.feedback

'YES'

# **Batch Evaluator**

In [38]:
from llama_index.core.evaluation import BatchEvalRunner

# Let's pick top 10 queries to do evaluation
batch_eval_queries = queries[:10]

# Initiate BatchEvalRunner to compute FaithFulness and Relevancy Evaluation.
runner = BatchEvalRunner(
    {"faithfulness": faithfulness_gpt4, "relevancy": relevancy_gpt4},
    workers=8,
)

# Compute evaluation
eval_results = await runner.aevaluate_queries(
    query_engine, queries=batch_eval_queries
)

In [39]:
# Let's get faithfulness score

faithfulness_score = sum(result.passing for result in eval_results['faithfulness']) / len(eval_results['faithfulness'])
faithfulness_score

1.0

In [40]:
# Let's get relevancy score

relevancy_score = sum(result.passing for result in eval_results['relevancy']) / len(eval_results['relevancy'])
relevancy_score


1.0